In [2]:
fun doStuff(): Int{
    return 123
}

print(doStuff())

123

In [10]:
import com.google.gson.Gson
import com.plaid.client.ApiClient
import com.plaid.client.model.PlaidError
import com.plaid.client.model.TransactionsGetRequest
import com.plaid.client.model.TransactionsGetResponse
import com.plaid.client.request.PlaidApi
import okhttp3.*
import okhttp3.MediaType.Companion.toMediaTypeOrNull
import java.io.File
import java.time.LocalDate

USE {
    dependencies {
//        implementation("com.plaid:plaid-java:20.1.0")
        implementation("com.plaid:plaid-java:8.2.0")
        
        implementation ("com.google.code.gson:gson:2.9.0")
        implementation("com.squareup.okhttp3:okhttp:4.9.1")
	    implementation("com.squareup.okhttp3:mockwebserver:4.9.1")
    }
}

val apiKeys = hashMapOf(
    "clientId" to "617975b96aaa7c0011b0a846",
    "secret" to "9cd3e5bd00e10c3a723f85c90d2f95",
)
val accessToken = "access-sandbox-84aa81b0-9c66-44f3-9443-338d54b49538"
val baseUrl = "https://sandbox.plaid.com"

fun <TResponse>callPlaid(func: () -> retrofit2.Response<TResponse> ): TResponse?{
    val response = func()
    if(!response.isSuccessful){
        val error = decodePlaidError(response)
        throw PlaidException(error)
    }
    val actualResponse = response.body()
    return actualResponse
}

class PlaidException(val plaidError: PlaidError) : Exception() {
    override val message: String
        get() = "Plaid Error ${plaidError.errorMessage}"
}

fun decodePlaidError(response: retrofit2.Response<*>): PlaidError{
    try{
        val gson = Gson()
        val error = gson.fromJson(response.errorBody()?.string(), PlaidError::class.java)
        return error
    } catch (e: Exception){
        throw Exception("failed converting error: $e")
    }
}

fun createClient(): PlaidApi {
    //use a transactions interceptor to read transactions from disk
    val okHttpClient = OkHttpClient.Builder()
        .addInterceptor(TransactionsInterceptor("../../src/test/resources/testdata/transactions.json"))
        .build()
//    val apiClient = ApiClient(apiKeys)
    val apiClient = ApiClient(okHttpClient)
    apiClient.setPlaidAdapter(baseUrl)
    val plaidClient = apiClient.createService(PlaidApi::class.java)
    return plaidClient
}

class TransactionsInterceptor(val filePath: String) : Interceptor {
    override fun intercept(chain: Interceptor.Chain): Response {
        val request = chain.request()
        val url = request.url.toString()
        val protocol = request.url.scheme
        // Check if the request URL matches the Plaid transactions endpoint
        if (url.contains("transactions/get")) {
            // Load the transactions.json file
            val jsonData = File(filePath).readText()
            return Response.Builder()
                .request(request)
//                .protocol(Protocol.get(protocol))
                .protocol(Protocol.HTTP_2)
                .code(200) // HTTP  200 OK
                .message("OK")
                .body(ResponseBody.create("application/json".toMediaTypeOrNull(), jsonData))
                .build()
        }

        // If the request doesn't match the endpoint, proceed with the original request
        return chain.proceed(request)
    }
}


In [11]:
getTransactions()

fun getTransactions(){
    val plaidClient = createClient()
    val request = TransactionsGetRequest()
    request.startDate = LocalDate.of(2023, 1, 1)
    request.endDate = LocalDate.of(2024, 3,1)
    request.accessToken = accessToken
    val transactionsResponse = callPlaid<TransactionsGetResponse> {
        plaidClient.transactionsGet(request).execute()
    }
    val transactions = transactionsResponse?.transactions

    assert(transactions?.size == 100)
    val transaction1 = transactions?.get(0)!!
    assert(transaction1.transactionId == "el49jjdkrPcNW7kBoeKjtmPk3d9jBmhr9e54V")
    println(transactionsResponse)
}

class TransactionsGetResponse {
    accounts: [class AccountBase {
        accountId: 77vG116MjafKq6mk83WEiW9m18Nq4BIdy8xEM
        balances: class AccountBalance {
            available: 100.0
            current: 110.0
            limit: null
            isoCurrencyCode: USD
            unofficialCurrencyCode: null
            lastUpdatedDatetime: null
        }
        mask: 0000
        name: Plaid Checking
        officialName: Plaid Gold Standard 0% Interest Checking
        type: depository
        subtype: checking
        verificationStatus: null
        verificationInsights: null
        persistentAccountId: 8cfb8beb89b774ee43b090625f0d61d0814322b43bff984eaf60386e
    }, class AccountBase {
        accountId: lxgwLLer49uMvkBP96NluvlEnmALkMipgaRKr
        balances: class AccountBalance {
            available: 200.0
            current: 210.0
            limit: null
            isoCurrencyCode: USD
            unofficialCurrencyCode: null
            lastUpdatedDatetime: null
